<a href="https://colab.research.google.com/github/YinGuoX/Deep_Learning_Pytorch_WithDeeplizard/blob/master/25_CNN_Output_Size_Formula_Bonus_Neural_Network_Debugging_Session.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN Output Size Formula - Tensor Transformations

在此情节中，我们将看到输入张量在流经CNN时是如何转换的。

## 神经网络设计概览
---
* 准备数据

* 建立模型

  * 了解前向传播的转换过程

* 训练模型

* 分析模型的结果

## 概览我们的神经网络模型
---

我们将使用的CNN是我们在过去一直使用的CNN。

* 输入层

* 隐藏的卷积层

* 隐藏的卷积层

* 隐藏的线性层

* 隐藏的线性层

* 输出层

我们使用PyTorch的nn.Module类构建了这个网络，Network类的定义如下：


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

In [ ]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)

        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)

    def forward(self, t):
        # (1) input layer
        t = t
        print("input layer shape:",t.shape)

        # (2) hidden conv layer
        t = self.conv1(t)
        print("conv1 layer shape:",t.shape)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        print("max_pool layer shape:",t.shape)
        # (3) hidden conv layer
        t = self.conv2(t)
        print("conv2 layer shape:",t.shape)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)

        # (4) hidden linear layer
        t = t.reshape(-1, 12 * 4 * 4)
        print("flatten layer shape:",t.shape)
        t = self.fc1(t)
        print("linear1 layer shape:",t.shape)
        t = F.relu(t)

        # (5) hidden linear layer
        t = self.fc2(t)
        print("linear2 layer shape:",t.shape)
        t = F.relu(t)

        # (6) output layer
        t = self.out(t)
        print("out layer shape:",t.shape)
        #t = F.softmax(t, dim=1)

        return t

In [ ]:
train_set = torchvision.datasets.FashionMNIST(root='./data',train=True,
                                download=True,
                                transform=transforms.Compose([
        transforms.ToTensor()
    ]))

In [ ]:
sample = next(iter(train_set))
image,label = sample

## 1.传递一批大小为一的图像（单个图像）

在上一集中，我们看到了如何使用PyTorch的unsqueeze（）方法添加批处理尺寸来传递单个图像。 我们将再次将该张量传递到网络，但是这次我们将使用调试器逐步处理forward（）方法。 这将使我们能够在执行变换时检查张量。

让我们开始：

In [ ]:
network = Network()
network(image.unsqueeze(0))

input layer shape: torch.Size([1, 1, 28, 28])
conv1 layer shape: torch.Size([1, 6, 24, 24])
max_pool layer shape: torch.Size([1, 6, 12, 12])
conv2 layer shape: torch.Size([1, 12, 8, 8])
flatten layer shape: torch.Size([1, 192])
linear1 layer shape: torch.Size([1, 120])
linear2 layer shape: torch.Size([1, 60])
out layer shape: torch.Size([1, 10])


tensor([[-0.0247,  0.1580,  0.1221,  0.1333, -0.1505, -0.0493, -0.0693, -0.1409,
          0.0318,  0.0665]], grad_fn=<AddmmBackward>)

### 1=输入层
---
当张量进入输入层时，我们有：


In [ ]:
t = image.unsqueeze(0)
t.shape

torch.Size([1, 1, 28, 28])

每个维度中的该值表示以下值：
* （批量大小、颜色通道、高度、宽度）

因为输入层只是标识函数，所以输出形状不会改变。

输入层可视为平凡的恒等函数，输出层等于输入层。

### 2=卷积层1
---
当张量进入这一层时，我们有：


```
> t.shape
torch.Size([1, 1, 28, 28])
```
在第一个卷积运算self.conv1之后，我们有：

```
> t.shape
torch.Size([1, 6, 24, 24])
```

批处理大小仍为1。这是有道理的，因为我们不希望更改批处理大小，在整个正向传递中都是如此。
* 在正向传播过程中，batch_size是固定的。

彩色通道的数量从1个增加到6个。当我们超越了第一个卷积层之后，我们不再认为通道是彩色通道了。我们只是把它们看作是输出通道。我们有6个输出通道的原因是在创建 self.conv1时指定了 out_channels的数量。

### 卷积运算使用过滤器
---
如我们所见，这个数字6是我们任意指定的。 out_channels参数指示nn.Conv2d图层类生成六个过滤器（也称为内核），形状为5×5，并具有随机初始化的值。 这些滤波器用于生成六个输出通道。
* out_channels参数确定将创建多少个过滤器。

过滤器是张量，当将张量传递到图层实例self.conv1时，它们用于对输入张量进行卷积。 滤波器张量内部的随机值是卷积层的权重。 但是请记住，实际上我们没有六个不同的张量。 所有六个过滤器都包装在一个高度和宽度为五个的单个权重张量中。

* 过滤器就是权重张量

使用权重张量（滤波器）对输入张量进行卷积后，结果就是输出通道。

**输出通道**的另一个名称是**特征映射**。 这里的术语是可以互换的。 这是由于以下事实：随着权重的更新而出现的图案检测代表了诸如边缘和其他更复杂图案的特征。

算法流程：

* 颜色通道已传入。

* 使用权重张量（滤波器）进行卷积。

* 生成特征图并将其继续前向传播。

从概念上讲，我们可以认为权重张量是不同的。 但是，我们在代码中真正拥有的是具有out_channels（过滤器）维的单个权重张量。 我们可以通过检查权重张量的形状来看到这一点：

In [ ]:
network.conv1.weight.shape

torch.Size([6, 1, 5, 5])

该张量的形状由下式给出：

* （过滤器数，输入通道数，过滤器高度，过滤器宽度）

### relu（）激活函数
---
对relu（）函数的调用将删除所有负值并将其替换为零。 我们可以通过在调用之前和之后检查张量的min（）来验证这一点。

In [ ]:
t = torch.tensor([-1,-1,-2,3])
t.min().item()

-2

In [ ]:
t = F.relu(t)
t.min().item()

0

relu（）函数可以用数学方式表示为:
$$ f(x)=\left\{
\begin{aligned}
0 & & if x<0 \\
x &  & if x>=0
\end{aligned}
\right.
$$

### 最大池操作
--- 
池化操作通过从张量中的每个2x2位置提取最大值来进一步减小张量的形状。


```
> t.shape
torch.Size([1, 6, 24, 24])
```


```

> t = F.max_pool2d(t, kernel_size=2, stride=2)
> t.shape
torch.Size([1, 6, 12, 12])
```



### 卷积层总结
---

卷积层输入和输出的张量的形状由下式给出：

* 输入形状：[1、1、28、28]

* 输出形状：[1、6、12、12]

发生的每个操作的总结：

* 卷积层使用六个随机初始化的5x5滤波器对输入张量进行卷积。

  * 这样可以将高度和宽度尺寸减少四倍。

* relu激活功能操作将所有负值映射为0。

  * 这意味着张量中的所有值现在都为正。

* 最大池化操作从由卷积创建的六个特征图的每个2x2部分中提取最大值。

  * 这样将高度和宽度尺寸减少了十二。

## 2.CNN输出大小公式

让我们看一下在执行卷积和池化操作之后计算张量的输出大小的公式。

### CNN输出大小公式（平方）
* 假设我们有一个n×n大小的输入。

* 假设我们有一个f×f大小的滤波器。

* 假设我们的填充为p，步幅为s。

* 输出大小O由以下公式给出：

$$
O=\frac{n-f+2p}{s}+1
$$

该值将输出的高度和宽度。 但是，如果输入或过滤器不是正方形，则需要两次应用此公式，一次用于宽度，一次用于高度。

### CNN输出大小公式（非平方）

* 假设我们有一个$n_h×n_w$大小的输入。

* 假设我们有一个$f_h×f_w$大小的滤波器。

* 假设我们的填充为p，步幅为s。

* 输出大小的高度$O_h$由以下公式给出：
$$
O_h = \frac{n_h-f_h+2p}{s}+1
$$
* 输出大小的宽度$O_w$由以下公式给出：
$$
O_w = \frac{n_w-f_w+2p}{s}+1
$$


## 3. 继续研究之后的卷积层操作
### 2=卷积层2
---
第二个隐藏的卷积层self.conv2在与self.conv1相同的条件下转换张量，并进一步减小了高度和宽度尺寸。 在进行这些转换之前，让我们检查一下self.conv2的权重张量的形状：

In [ ]:
network.conv2.weight.shape

torch.Size([12, 6, 5, 5])

这一次我们的权重张量有12个过滤器，高度为5，宽度为5，但是没有一个单一的输入通道，通道的数量是6，这给了过滤器一个深度。这就解释了第一卷积层的六个输出通道。结果输出将有12个通道。

现在让我们运行这些操作


```
> t.shape
torch.Size([1, 6, 12, 12])

> t = self.conv2(t)
> t.shape
torch.Size([1, 12, 8, 8])

> t.min().item()
-0.39324113726615906

> t = F.relu(t)
> t.min().item()
0.0

> t = F.max_pool2d(t, kernel_size=2, stride=2)
> t.shape
torch.Size([1, 12, 4, 4])
```




self.conv2输出结果的形状使我们了解了为什么在将张量传递到第一个线性层self.fc1之前使用12 * 4 * 4重整张量。

正如我们在过去所看到的，这种特殊的重塑称为拉平张量。 展平操作将所有张量元素置于一个维中。



```
> t = t.reshape(-1, 12*4*4)
> t.shape
torch.Size([1, 192])
```
产生的形状为1x192。 在这种情况下，1表示批处理大小，而192表示张量中现在处于相同维度的元素数。



### 4,5,6的线性层
现在，我们只有一系列线性层，然后是非线性激活函数，直到到达输出层。



```
> t = self.fc1(t)

> t.shape
torch.Size([1, 120])

> t = self.fc2(t)
> t.shape
torch.Size([1, 60])

> t = self.out(t)
> t.shape
torch.Size([1, 10])

> t
tensor([[ 0.1009, -0.0842,  0.0349, -0.0640,  0.0754, -0.0057,  0.0878,  0.0296,  0.0345,  0.0236]])
```

下表总结了形状更改操作以及每种形状的结果形状：



| Operation	| Output | Shape |
| :----: | :----: | :----: |
| Identity | function |	torch.Size([1, 1, 28, 28]) |
| Convolution | (5 x 5)	| torch.Size([1, 6, 24, 24]) |
| Max pooling | (2 x 2)	| torch.Size([1, 6, 12, 12]) | 
| Convolution | (5 x 5)	| torch.Size([1, 12, 8, 8])|
|Max pooling |(2 x 2)	|torch.Size([1, 12, 4, 4])|
|Flatten |(reshape)	|torch.Size([1, 192])|
|Linear |transformation	|torch.Size([1, 120])|
|Linear |transformation	|torch.Size([1, 60])|
|Linear |transformation	|torch.Size([1, 10])|

## 下一步是训练CNN

我们现在应该对卷积神经网络如何转换输入张量，如何在PyTorch中调试神经网络，以及如何检查所有层的权重张量有很好的了解。

在节中，我们将开始训练网络，这将导致我们的权重张量值得到更新，以使网络的正向方法将输入映射到正确的输出类。 下次见！